In [1]:
import json
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# loading the data
with open('../Final_Intents.json', 'r') as file:
    data = json.load(file)

In [3]:
# Function to pair each question with its corresponding response
def pair_questions_responses(data):
    paired_data = []
    for item in data:
        tag = item.get('tag', 'Unknown')
        questions = item.get('questions', [])
        responses = item.get('responses', [])
        
        for question, response in zip(questions, responses):
            paired_data.append({'tag': tag, 'question': question, 'response': response})
    
    return paired_data

In [4]:
paired_data = pair_questions_responses(data)

In [5]:
df = pd.DataFrame(paired_data)
df.head(20)

,tag,question,response
0,cloud_computing,What is the fundamental starting point for und...,The fundamental starting point for understandi...
1,cloud_computing,How does Moringa School introduce the concept ...,Moringa School introduces the concept of Cloud...
2,cloud_computing,Why is Cloud Computing considered essential in...,Cloud Computing is considered essential in the...
3,cloud_computing,Can you provide an overview of the Cloud Compu...,The Cloud Computing program at Moringa School ...
4,cloud_computing,What are the eligibility requirements for the ...,To be eligible for the Cloud Computing program...
5,cloud_computing,How long do students need to dedicate to succe...,Students are required to dedicate 3 days to le...
6,cloud_computing,What unique learning experiences does Moringa ...,Moringa School offers a unique learning experi...
7,cloud_computing,How is the curriculum at Moringa School tailor...,The curriculum at Moringa School is tailored t...
8,cloud_computing,What specific areas does the Cloud Computing c...,The Cloud Computing course covers various area...
9,cloud_computing,In what ways does Moringa School guarantee a c...,Moringa School guarantees a comprehensive unde...


In [6]:
df.info ()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599 entries, 0 to 598
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tag       599 non-null    object
 1   question  599 non-null    object
 2   response  599 non-null    object
dtypes: object(3)
memory usage: 14.2+ KB


In [7]:
# Data Preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
def preprocess_text(text):
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return " ".join(words)

# Apply preprocessing to questions and responses
df['question'] = df['question'].apply(preprocess_text)
df['response'] = df['response'].apply(preprocess_text)

# Split data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['question'].tolist() + train_data['response'].tolist())

# Convert texts to sequences
train_questions_seq = tokenizer.texts_to_sequences(train_data['question'].tolist())
train_responses_seq = tokenizer.texts_to_sequences(train_data['response'].tolist())

# Finding the maximum sequence length
max_seq_length = max(max(len(seq) for seq in train_questions_seq), max(len(seq) for seq in train_responses_seq))

# Padding sequences
train_questions_padded = pad_sequences(train_questions_seq, maxlen=max_seq_length, padding='post')
train_responses_padded = pad_sequences(train_responses_seq, maxlen=max_seq_length, padding='post')

# Preparing test data
test_questions_seq = tokenizer.texts_to_sequences(test_data['question'].tolist())
test_responses_seq = tokenizer.texts_to_sequences(test_data['response'].tolist())
test_questions_padded = pad_sequences(test_questions_seq, maxlen=max_seq_length, padding='post')
test_responses_padded = pad_sequences(test_responses_seq, maxlen=max_seq_length, padding='post')

# One-hot encode the responses
vocab_size = len(tokenizer.word_index) + 1
train_responses_one_hot = np.zeros((len(train_responses_padded), max_seq_length, vocab_size))
for i, sequence in enumerate(train_responses_padded):
    for j, word_index in enumerate(sequence):
        train_responses_one_hot[i, j, word_index] = 1

In [8]:
from tensorflow.keras.layers import Layer
import tensorflow as tf
from tensorflow.keras.layers import Bidirectional, Attention, Concatenate

class BahdanauAttention(Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

In [ ]:
history = model.fit([train_questions_padded, train_responses_padded], train_responses_one_hot,
                    batch_size=64,
                    epochs=20,
                    validation_split=0.2)

In [ ]:
# One-hot encode the test responses
test_responses_one_hot = np.zeros((len(test_responses_padded), max_seq_length, vocab_size))
for i, sequence in enumerate(test_responses_padded):
    for j, word_index in enumerate(sequence):
        test_responses_one_hot[i, j, word_index] = 1

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate([test_questions_padded, test_responses_padded], test_responses_one_hot)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
train_loss, train_accuracy = model.evaluate([train_questions_padded, train_responses_padded], train_responses_one_hot)
print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)

In [ ]:
train_loss, train_accuracy = model.evaluate([train_questions_padded, train_responses_padded], train_responses_one_hot)
print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)

In [ ]:
def seq_to_text(sequence, tokenizer):
    # Convert a sequence of indices back to text
    return " ".join([word for word, index in tokenizer.word_index.items() if index in sequence])

In [ ]:
test_predictions = model.predict([test_questions_padded, test_responses_padded])

# Convert one-hot encoded predictions back to text
predicted_texts = []
actual_texts = []

for i in range(len(test_predictions)):
    predicted_sequence = np.argmax(test_predictions[i], axis=-1)
    predicted_text = seq_to_text(predicted_sequence, tokenizer)
    actual_text = seq_to_text(test_responses_padded[i], tokenizer)
    
    predicted_texts.append(predicted_text)
    actual_texts.append(actual_text)

# Now you can calculate BLEU and ROUGE scores
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge 

# BLEU Score
hypothesis = [text.split() for text in predicted_texts]
references = [[text.split()] for text in actual_texts]
bleu_score = corpus_bleu(references, hypothesis)
print(f"BLEU Score: {bleu_score}")

# ROUGE Score
rouge = Rouge()
rouge_scores = rouge.get_scores(predicted_texts, actual_texts, avg=True)
print(f"ROUGE Scores: {rouge_scores}")